In [38]:
BLOCK_PATH = '../data/Blocks.csv'
WELL_PATH = '../data/Wells.csv'
PROSPECT_PATH = '../data/Prospects.csv'
CYPHER_TEXT = '../data/Cypher.txt'

import pandas as pd

BLOCK = pd.read_csv(BLOCK_PATH, sep=';')
WELL = pd.read_csv(WELL_PATH, sep=';')
PROSPECT = pd.read_csv(PROSPECT_PATH, sep=';')

In [39]:
PROSPECT

,ProspectID,BlockID,ProspectDescription,ProsepectOperator,ProspectAbbr,ProspectName,ProsepectCode,ProspectClass
0,P1,CL39,Agate is a dual culmination anticlinal drape o...,PETRONAS,01-E,Agate,BC_PNAS01_02_694,Prospect
1,P10,CL39,The Ruby prospect is located in the southweste...,PETRONAS,01-B,Ruby,BC_PNAS01_02_004,Field
2,P100,CL10,NaN,PHU_QUY_JOC,NaN,Gau Den-South,BC_PHUQUY.POC15.1/05_002,Prospect
3,P101,CL5,NaN,PHU_QUY_JOC,NaN,Gau Trang,BC_PHUQUY.POC15.1/05_002,Prospect
4,P102,CL10,NaN,PHU_QUY_JOC,NaN,Gau Xam,BC_PHUQUY.POC15.1/05_002,Prospect
...,...,...,...,...,...,...,...,...
260,P95,CL10,Lac Da Vang (LDV) Cluster locates in the south...,PHU_QUY_JOC,NaN,Lac Da Vang,BC_PHUQUY.POC15.1/05_037,Prospect
261,P96,NaN,NaN,PHU_QUY_JOC,NaN,Lac Da Xam,BC_PHUQUY.POC15.1/05_044,Prospect
262,P97,CL10,The northwestern of the Block 15-1/05.,PHU_QUY_JOC,NaN,La Da Den,BC_PHUQUY.POC15.1/05_044,Prospect
263,P98,CL10,The northwestern of the Block 15-1/05.,PHU_QUY_JOC,NaN,Lac Da Trang,BC_PHUQUY.POC15.1/05_044,Prospect


In [40]:
WELL['WellAlias'] = WELL['WellAlias'].apply(lambda x: x.replace(' (', '_') if type(x) == str else x)
WELL['WellAlias'] = WELL['WellAlias'].apply(lambda x: x.replace(')', '_') if type(x) == str else x)
WELL['WellName'] = WELL['WellName'].apply(lambda x: x.replace(' (', '_') if type(x) == str else x)
WELL['WellName'] = WELL['WellName'].apply(lambda x: x.replace(')', '_') if type(x) == str else x)
WELL['WellUWI'] = WELL['WellUWI'].apply(lambda x: x.replace(' (', '_') if type(x) == str else x)
WELL['WellUWI'] = WELL['WellUWI'].apply(lambda x: x.replace(')', '_') if type(x) == str else x)

# Using lambda function to replace , by space in GeoJson
BLOCK['BlockDescription'] = BLOCK['BlockDescription'].apply(lambda x: x.replace(',', ' ') if type(x) == str else x)
BLOCK['BlockArea'] = BLOCK['BlockArea'].apply(lambda x: x.replace(',', '') if type(x) == str else x)

PROSPECT['ProspectDescription'] = PROSPECT['ProspectDescription'].apply(lambda x: x.replace("'", '') if type(x) == str else x)
PROSPECT['ProspectDescription'] = PROSPECT['ProspectDescription'].apply(lambda x: x.replace('"', '') if type(x) == str else x)

In [41]:

cypher_txt = ""
for i in range(len(BLOCK)):
    create_block_query = "MERGE (p:Block {{name: '{}', id: '{}', description: '{}', status: '{}', area: '{}', geojson: '{}' }})".format(
        BLOCK['BlockName'][i], 
        BLOCK['BlockID'][i], 
        BLOCK['BlockDescription'][i], 
        BLOCK['BlockStatus'][i], 
        BLOCK['BlockArea'][i], 
        BLOCK['GeoJson'][i], 
        )
    cypher_txt += create_block_query + '\n'
for i in range(len(WELL)):
    create_well_query = "MERGE (p:Well {{name: '{}', id: '{}', field_id: '{}', operator_id: '{}', well_alias: '{}', well_type: '{}', full_name: '{}', geojson: '{}' }})".format(
        WELL['WellUWI'][i], 
        WELL['Well_index'][i], 
        WELL['FieldID'][i], 
        WELL['OperatorID'][i], 
        WELL['WellAlias'][i], 
        WELL['WellType'][i], 
        WELL['WellName'][i], 
        WELL['GeoJson'][i], 
        )
    cypher_txt += create_well_query + '\n'
    
for i in range(len(PROSPECT)):
    create_prospect_query = "MERGE (p:Prospect {{name: '{}', id: '{}', description: '{}', operator_id: '{}', class: '{}'}})".format(
        PROSPECT['ProspectName'][i], 
        PROSPECT['ProspectID'][i], 
        PROSPECT['ProspectDescription'][i], 
        PROSPECT['ProsepectOperator'][i], 
        PROSPECT['ProspectClass'][i], 
        )
    cypher_txt += create_prospect_query + '\n'

# Add text to Cypher.txt
with open(CYPHER_TEXT, 'w') as f:
    f.write(cypher_txt)

In [16]:
relations = pd.read_csv('../data/relations.csv', sep=",")
relations

# WELL - PROSPECT
for i in range(len(relations)):
    create_r1 = """
    MATCH (s:{} {{name: '{}'}}), (o:{} {{id: '{}'}})
    CREATE (s)-[:KHOAN_VÀO_ĐỐI_TƯỢNG]->(o)
    """.format(
        relations['WellAlias'][i], 
        relations['ProspectID'][i], 
        )

,WellAlias,OperatorID,ProspectID,BlockID,BlockName
0,15-2-RD-E-25P,JVPC,P110,CL12,15-2
1,09-2-CNV-5PST2,HOAN_VU_JOC,P51,CL6,09-2
2,SNS-5AP,CUU_LONG_JOC,P249,CL9,15-1
3,09-2-CNV-2P (RE),HOAN_VU_JOC,P51,CL6,09-2
4,09-2-CNV-2P,HOAN_VU_JOC,P51,CL6,09-2
...,...,...,...,...,...
605,01/97-HXS-1X,LAM_SON_JOC,P17,CL1,01/97
606,JADE-3X,PCVL,P26,CL40,NaN
607,Azurite 2X,PCVL,P5,CL39,NaN
608,01/97-HD-1X,LAM_SON_JOC,P18,CL1,01/97
